#### План
1) Каждый столбец преобразовать в нужный тип данных
2) Соеденить кажждый столбец для получения тренировочной выборки
3) Сделать тестовую выборку
4) Сделать колонки средний возраст и средний пол по url

#### Наблюдения
1. <b>Столбец price содержит</b> 6 589 482 пропуска если убрать пропуски можно пребразовать в int32
2. <b>Названия столбцов</b> - 'region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
       'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price', 'date',
       'part_of_day', 'request_cnt', 'user_id'
3. <b>словарь с типами данных</b>: 'region_name': 'category', 'city_name': 'category', 'cpe_manufacturer_name': 'category', 'cpe_model_name': 'category', 'url_host': 'object', 'cpe_type_cd':'category', 'cpe_model_os_type': 'category', 'price': 'float64', 'date': 'object', 'part_of_day': 'category', 'request_cnt': 'int8', 'user_id': 'int32', 'is_male': 'float64', 'age':'int8', 'group':'float64'

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import dask.dataframe as dd
#import pyarrow.parquet as pq
from IPython.display import clear_output
import time
import bisect

In [4]:
%%time
features_path = '.\\competition_data_final_pqt'
name_column = 'user_id'
data_submit = dd.read_parquet('submit_2.pqt').compute()

data = None
for i in range(10):
    data_features_file = dd.read_parquet(features_path + f'\\part-0000{i}-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet')

    if i == 0:
        data = data_features_file.merge(data_submit[['user_id']], how='inner', on='user_id').compute()
    else:
        data = pd.concat([data, data_features_file.merge(data_submit[['user_id']], how='inner', on='user_id').compute()], ignore_index=True)

    del data_features_file

data.to_parquet('Dataset\\train test\\test_data_pqt.parquet', index=False)

print(data.info(verbose=True, show_counts=True))
print(len(data['user_id'].unique()))
print(data.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111734107 entries, 0 to 111734106
Data columns (total 12 columns):
 #   Column                 Non-Null Count      Dtype  
---  ------                 --------------      -----  
 0   region_name            111734107 non-null  object 
 1   city_name              111734107 non-null  object 
 2   cpe_manufacturer_name  111734107 non-null  object 
 3   cpe_model_name         111734107 non-null  object 
 4   url_host               111734107 non-null  object 
 5   cpe_type_cd            111734107 non-null  object 
 6   cpe_model_os_type      111734107 non-null  object 
 7   price                  109416393 non-null  float64
 8   date                   111734107 non-null  object 
 9   part_of_day            111734107 non-null  object 
 10  request_cnt            111734107 non-null  int64  
 11  user_id                111734107 non-null  int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 10.0+ GB
None
144724
          region_name  c

In [3]:
print(len(data_submit['user_id'].unique()))

144724


In [13]:
%%time
data_all = data_all.astype("int32")
print(data_all.memory_usage()/1024/1024)

data_all = pd.DataFrame(data = data_all.values, columns=[f'{name_column}'])
data_all.to_parquet('.\\Dataset' + f'\\{name_column}.parquet', index=False)
del data_all

1231.76371383667
CPU times: total: 3.81 s
Wall time: 3.84 s


In [20]:
data_all = pd.DataFrame()

for column in ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
               'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price', 'date',
               'part_of_day', 'request_cnt', 'user_id']:
    data_all[column] = dd.read_parquet('.\\Dataset' + f'\\{column}.parquet').compute()[column]

data_all.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 12 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   region_name            category
 1   city_name              category
 2   cpe_manufacturer_name  category
 3   cpe_model_name         category
 4   url_host               object  
 5   cpe_type_cd            category
 6   cpe_model_os_type      category
 7   price                  float64 
 8   date                   object  
 9   part_of_day            category
 10  request_cnt            int8    
 11  user_id                int32   
dtypes: category(7), float64(1), int32(1), int8(1), object(2)
memory usage: 11.4+ GB


In [21]:
data_all.to_parquet('.\\Dataset' + f'\\full_data.parquet', index=False)

In [24]:
#data_target_file = dd.read_parquet('public_train.pqt').compute()

In [2]:
%%time
data_all = dd.read_parquet('.\\Dataset' + f'\\full_data.parquet').compute()
data_all.to_csv('.\\Dataset' + f'\\full_data.csv', chunksize=100000)
del data_all

CPU times: total: 24min 7s
Wall time: 27min 6s


### Создаём тренировочный датасет с колонками пол возраст и группа возраста

In [14]:
%%time
''' Создаём тренировочный датасет с колонками пол возраст и группа возраста '''
#Не забываем удалять файл если он уже есть
header = True

data_target_file = dd.read_parquet('public_train.pqt').compute()

for data_chink in pd.read_csv('.\\Dataset' + f'\\full_data.csv', chunksize=100000):
    #print(data_chink.drop(['Unnamed: 0'], axis = 1))
    data_chink = data_chink.drop(['Unnamed: 0'], axis = 1)

    data_chink = data_chink.merge(data_target_file[['age', 'is_male','user_id']], on = 'user_id', how = 'left')#соединяем столбцы с полом и возрастом
    data_chink = data_chink[data_chink['age'].notna()] #Берём без пропусков
    data_chink["group"] = np.nan

    data_chink.loc[data_chink['age']<18, ['group']] = 0
    data_chink.loc[(18<=data_chink['age']) & (data_chink['age']<=24), ['group']] = 1
    data_chink.loc[(25<=data_chink['age']) & (data_chink['age']<=34), ['group']] = 2
    data_chink.loc[(35<=data_chink['age']) & (data_chink['age']<=44), ['group']] = 3
    data_chink.loc[(45<=data_chink['age']) & (data_chink['age']<=54), ['group']] = 4
    data_chink.loc[(55<=data_chink['age']) & (data_chink['age']<=64), ['group']] = 5
    data_chink.loc[65<=data_chink['age'], ['group']] = 6

    data_chink['group'] = data_chink['group'].astype('int8')
    data_chink['age'] = data_chink['age'].astype('int8')

    data_chink.to_csv('.\\Dataset\\train test' + f'\\train_data.csv', header=header, mode='a', index=False)

    header = False

del data_target_file
#data_all = data_all.merge(data_target_file[['age', 'is_male','user_id']], on = 'user_id', how = 'left')
#data_all = data_all[data_all['age'].notna()]
#data_all.info()

CPU times: total: 28min 48s
Wall time: 29min 4s


In [2]:
%%time
'''
for data_chink in pd.read_csv('.\\Dataset\\train test\\train_data.csv', chunksize=100000):
    print(data_chink[data_chink['is_male'].isna()])
    data_chink.info()
    break
'''
#читаем с правильным типом
type_col = {'region_name': 'category', 'city_name': 'category', 'cpe_manufacturer_name': 'category', 'cpe_model_name': 'category',
            'url_host': 'object', 'cpe_type_cd':'category', 'cpe_model_os_type': 'category', 'price': 'float64', 'date': 'object',
            'part_of_day': 'category', 'request_cnt': 'int8', 'user_id': 'int32', 'is_male': 'float64', 'age':'int8', 'group':'float64'}

data_all = dd.read_csv('.\\Dataset\\train test\\train_data.csv',  dtype=type_col).compute()

#usecols = ['city_name', 'cpe_manufacturer_name', 'cpe_model_name','url_host', 'cpe_type_cd', 'price','part_of_day', 'request_cnt', 'is_male'],

In [3]:
data_all.info(show_counts=True)
#del data_all

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210730009 entries, 0 to 436949
Data columns (total 15 columns):
 #   Column                 Non-Null Count      Dtype   
---  ------                 --------------      -----   
 0   region_name            210730009 non-null  category
 1   city_name              210730009 non-null  category
 2   cpe_manufacturer_name  210730009 non-null  category
 3   cpe_model_name         210730009 non-null  category
 4   url_host               210560584 non-null  object  
 5   cpe_type_cd            210730009 non-null  category
 6   cpe_model_os_type      210730009 non-null  category
 7   price                  206464205 non-null  float64 
 8   date                   210730009 non-null  object  
 9   part_of_day            210730009 non-null  category
 10  request_cnt            210730009 non-null  int8    
 11  user_id                210730009 non-null  int32   
 12  age                    210730009 non-null  int8    
 13  is_male                206

In [4]:
data_all.to_parquet('.\\Dataset\\train test\\train_data_pqt.parquet', index=False)

In [ ]:
# посмотреть сколько в среднем срок у одного пользователя

In [5]:
del data_all

### Тестовая выборка для отправки

In [13]:
%%time
#Не забываем удалять файл если он уже есть
header = True

data_target_file = dd.read_parquet('submit_2.pqt').compute()

for data_chink in pd.read_csv('.\\Dataset' + f'\\full_data.csv', chunksize=100000):
    #print(data_chink.drop(['Unnamed: 0'], axis = 1))
    data_chink = data_chink.drop(['Unnamed: 0'], axis = 1)

    data_chink = data_chink.merge(data_target_file[['user_id']], on = 'user_id', how = 'inner')#соединяем столбцы с полом и возрастом
    #print('Итог: ', data_chink)
    #print(data_target_file[data_target_file['user_id']==45098])
    '''
    if header == False:
        break
    '''
    data_chink.to_csv('.\\Dataset\\train test' + f'\\test_data_sbm.csv', header=header, mode='a', index=False)
    header = False


del data_target_file

CPU times: total: 17min 58s
Wall time: 18min 3s


In [14]:
%%time
type_col = {'region_name': 'category', 'city_name': 'category', 'cpe_manufacturer_name': 'category', 'cpe_model_name': 'category',
            'url_host': 'object', 'cpe_type_cd':'category', 'cpe_model_os_type': 'category', 'price': 'float64', 'date': 'object',
            'part_of_day': 'category', 'request_cnt': 'int8', 'user_id': 'int32'}

data_all = dd.read_csv('.\\Dataset\\train test\\test_data_sbm.csv',  dtype=type_col).compute()

print(data_all.info(show_counts=True))

data_all.to_parquet('.\\Dataset\\train test\\test_data_pqt.parquet', index=False)

del data_all

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111734107 entries, 0 to 462112
Data columns (total 12 columns):
 #   Column                 Non-Null Count      Dtype   
---  ------                 --------------      -----   
 0   region_name            111734107 non-null  category
 1   city_name              111734107 non-null  category
 2   cpe_manufacturer_name  111734107 non-null  category
 3   cpe_model_name         111734107 non-null  category
 4   url_host               111640766 non-null  object  
 5   cpe_type_cd            111734107 non-null  category
 6   cpe_model_os_type      111734107 non-null  category
 7   price                  109416393 non-null  float64 
 8   date                   111734107 non-null  object  
 9   part_of_day            111734107 non-null  category
 10  request_cnt            111734107 non-null  int8    
 11  user_id                111734107 non-null  int32   
dtypes: category(7), float64(1), int32(1), int8(1), object(2)
memory usage: 4.8+ GB
None

#### Сделаем 2 столбца с наиболее часто встречающимеся значениями

In [2]:
%%time
data_all = dd.read_parquet('.\\Dataset\\train test\\train_data_pqt.parquet').compute()
print(data_all.info())
#, columns=[]
#'region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name','url_host', 'price', 'user_id'
#'cpe_type_cd', 'cpe_model_os_type', 'date','part_of_day', 'request_cnt', 'age', 'is_male', 'user_id'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210730009 entries, 0 to 210730008
Data columns (total 15 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   region_name            category
 1   city_name              category
 2   cpe_manufacturer_name  category
 3   cpe_model_name         category
 4   url_host               object  
 5   cpe_type_cd            category
 6   cpe_model_os_type      category
 7   price                  float64 
 8   date                   object  
 9   part_of_day            category
 10  request_cnt            int8    
 11  user_id                int32   
 12  age                    int8    
 13  is_male                float64 
 14  group                  float64 
dtypes: category(7), float64(3), int32(1), int8(2), object(2)
memory usage: 10.8+ GB
None
CPU times: total: 4min 52s
Wall time: 7min 19s


In [3]:
data_all['group'] = data_all['group'].astype('int8')
data_all['date'] = data_all['date'].astype('datetime64[ns]')
#data_all.info(show_counts=True)

In [13]:
print(data_all[data_all['user_id']==data_all['user_id'][1]]['cpe_model_name'].value_counts(), end='\n\n')
print('Чаще всего = ', data_all[data_all['user_id']==data_all['user_id'][1]]['cpe_model_name'].mode().values)

Galaxy S20+                   1635
ZenFone 3 Max ZC553KL Dual       0
Blade A6                         0
L6006                            0
M6 Note                          0
                              ... 
Galaxy S9 Dual                   0
Honor 8C                         0
MI 5S                            0
P20 Pro                          0
Power Five Max                   0
Name: cpe_model_name, Length: 599, dtype: int64

Чаще всего =  ['Galaxy S20+']
Categories (599, object): ['Galaxy A10 Dual', 'Galaxy A21s', 'Galaxy A30 Dual', 'Galaxy A40 Dual', ..., '16', 'Xperia XA', 'Redmi Note 5A Lite', 'Power Five Max']


In [4]:
unique_id = pd.unique(data_all['user_id'])
#unique_url = pd.unique(data_all['url_host'])
print(len(unique_id))

269999


#### Сделаем новую таблицу с уникальными id и с самыми популярными url и тд

In [2]:
data_target = dd.read_parquet('public_train.pqt').compute()
#data_target = dd.read_parquet('submit_2.pqt').compute()

#data_target['user_id'].unique()

In [3]:
%%time

def func(x):
    try:
        x = x.value_counts().idxmax()
    except:
        x = np.nan
    return x

features_path = '.\\competition_data_final_pqt'
name_column = 'user_id'

dict_dtype = {'region_name': 'category', 'city_name': 'category', 'cpe_manufacturer_name': 'category', 'cpe_model_name': 'category','url_host': 'object', 'cpe_type_cd':'category', 'cpe_model_os_type': 'category', 'price': 'float64', 'date': 'object','part_of_day': 'category', 'request_cnt': 'int8', 'user_id': 'int32'}


#print(dict_dtype['region_name'])
for i in range(10):
    start_time = time.time()
    print(i)

    data_features = dd.read_parquet(features_path + f'\\part-0000{i}-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet').compute()
    for col in data_features.columns:
        data_features[col] = data_features[col].astype(dict_dtype[col])

    data_features = data_features.merge(data_target, on = 'user_id', how = 'inner')
    #соединяем столбцы с полом и возрастом

    #print(data_features.info(show_counts=True))
    col_name = list(data_features.columns)#
    col_name.remove('user_id')

    table = pd.pivot_table(data_features, index='user_id', values=col_name, aggfunc=func)
    table['user_id'] = table.index

    #table.to_csv(f'.\\Dataset\\csv train\\train_{i}.csv', header=True, index=False)
    table.to_csv(f'.\\Dataset\\csv test\\test_{i}.csv', header=True, index=False)

    print('Время прошло: ', time.time() - start_time)
    del data_features

0
Время прошло:  169.75897240638733
1
Время прошло:  157.69882917404175
2
Время прошло:  151.01837182044983
3
Время прошло:  155.82675099372864
4
Время прошло:  146.77883386611938
5
Время прошло:  147.04509234428406
6
Время прошло:  145.18527793884277
7
Время прошло:  148.16920566558838
8
Время прошло:  146.1513488292694
9
Время прошло:  146.70565032958984
CPU times: total: 24min 53s
Wall time: 25min 15s


In [4]:
#del data_features

data_all = pd.DataFrame()
for i in range(10):
    #df_buf = pd.read_csv(f'.\\Dataset\\csv train\\train_{i}.csv')
    df_buf = pd.read_csv(f'.\\Dataset\\csv test\\test_{i}.csv')
    data_all = pd.concat([data_all, df_buf], ignore_index=True)
    print(df_buf.shape)

data_all.to_csv(f'.\\Dataset\\train test\\test_data_unique.csv', header=True, index=False)

(14368, 12)
(14463, 12)
(14366, 12)
(14572, 12)
(14315, 12)
(14426, 12)
(14485, 12)
(14616, 12)
(14623, 12)
(14490, 12)


In [3]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270000 entries, 0 to 269999
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   age                    269999 non-null  float64
 1   city_name              270000 non-null  object 
 2   cpe_manufacturer_name  270000 non-null  object 
 3   cpe_model_name         270000 non-null  object 
 4   cpe_model_os_type      270000 non-null  object 
 5   cpe_type_cd            270000 non-null  object 
 6   date                   270000 non-null  object 
 7   is_male                264326 non-null  float64
 8   part_of_day            270000 non-null  object 
 9   price                  263078 non-null  float64
 10  region_name            270000 non-null  object 
 11  request_cnt            270000 non-null  int64  
 12  url_host               269923 non-null  object 
 13  user_id                270000 non-null  int64  
dtypes: float64(3), int64(2), object(9)
m

### Метрика маленькая, поэтому мы делаем таблицу с веротностями сначала для пола по категориальным столбцам, потом для возраста

In [ ]:
b = None
def test():
    print(b)

for i in range(3):
    b = i
    test()

In [2]:
%%time

col_features = ''

def func(column): #Исправить с названием колонки
    try:
        row = (table_features[table_features[col_features].isin(column.unique())]['sum']['is_male'] / table_features[table_features[col_features].isin(column.unique())]['count']['is_male']).mean()
        #print(row)
    except:
        row = np.nan
    return row
#'city_name', 'cpe_manufacturer_name', 'cpe_model_name', 'cpe_type_cd', 'part_of_day', 'url_host',
column_name = [ 'cpe_model_os_type', 'region_name'] #, 'region_name' уже сделал

for col in column_name:
    time_start = time.time()

    data_all = dd.read_parquet('.\\Dataset\\train test\\train_data_pqt.parquet', columns=[col,'user_id', 'is_male']).compute()
    print(data_all.info(show_counts=True))

    col_features = col #Делаем переменную для признака

    table_features = pd.pivot_table(data_all, index=col, values='is_male', aggfunc=[np.sum, 'count'])
    table_features[col] = table_features.index
    table_features.to_csv(f'.\\Dataset\\prob_col_csv\\prob_{col}.csv', header=True, index=False)

    table = pd.pivot_table(data_all, index='user_id', values=col, aggfunc=func)
    table['user_id'] = table.index

    table.to_csv(f'.\\Dataset\\prob_train_csv\\train_{col}.csv', header=True, index=False)
    print('\n\nВремени прошло: ', time.time() - time_start, '\n\n')
    del data_all
    del table_features
    del table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210730009 entries, 0 to 210730008
Data columns (total 3 columns):
 #   Column             Non-Null Count      Dtype   
---  ------             --------------      -----   
 0   cpe_model_os_type  210730009 non-null  category
 1   user_id            210730009 non-null  int32   
 2   is_male            206839343 non-null  float64 
dtypes: category(1), float64(1), int32(1)
memory usage: 2.6 GB
None


Времени прошло:  656.5586478710175 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210730009 entries, 0 to 210730008
Data columns (total 3 columns):
 #   Column       Non-Null Count      Dtype   
---  ------       --------------      -----   
 0   region_name  210730009 non-null  category
 1   user_id      210730009 non-null  int32   
 2   is_male      206839343 non-null  float64 
dtypes: category(1), float64(1), int32(1)
memory usage: 2.6 GB
None


Времени прошло:  647.1826252937317 


CPU times: total: 21min 33s
Wall time: 21min 43s


In [3]:
%%time
table_reg_name = pd.pivot_table(data_all, index='region_name', values='is_male', aggfunc=[np.sum, 'count'])
table_reg_name['region_name'] = table_reg_name.index

In [4]:
%%time
def func(column):
    try:
        row = (table_reg_name[table_reg_name['region_name'].isin(column.unique())]['sum']['is_male'] / table_reg_name[table_reg_name['region_name'].isin(column.unique())]['count']['is_male']).mean()
        #print(row)
    except:
        row = np.nan
    return row

table = pd.pivot_table(data_all, index='user_id', values='region_name', aggfunc=func)


CPU times: total: 12min 30s
Wall time: 12min 32s


,region_name
user_id,
0,0.579263
1,0.553181
2,0.514781
3,0.581428
4,0.495097
...,...
415308,0.579263
415309,0.515158
415311,0.395121


In [9]:
table['user_id'] = table.index
table.to_csv(f'.\\Dataset\\prob_train_csv\\train_region_name.csv', header=True, index=False)

##### Пробуем другой метод для группировки

In [2]:
%%time
#del data_features
data_target = dd.read_parquet('public_train.pqt').compute()
dict_dtype = {'region_name': 'category', 'city_name': 'category', 'cpe_manufacturer_name': 'category', 'cpe_model_name': 'category','url_host': 'object', 'cpe_type_cd':'category', 'cpe_model_os_type': 'category', 'price': 'float64', 'date': 'object','part_of_day': 'category', 'request_cnt': 'int8', 'user_id': 'int32'}

data_features = dd.read_parquet('.\\competition_data_final_pqt\\part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet').compute()

for col in data_features.columns:
    data_features[col] = data_features[col].astype(dict_dtype[col])

data_features = data_features.merge(data_target, on = 'user_id', how = 'inner')#соединяем столбцы с полом и возрастом


data_features = data_features.loc[:100000]
data_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100001 entries, 0 to 100000
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   region_name            100001 non-null  category
 1   city_name              100001 non-null  category
 2   cpe_manufacturer_name  100001 non-null  category
 3   cpe_model_name         100001 non-null  category
 4   url_host               100001 non-null  object  
 5   cpe_type_cd            100001 non-null  category
 6   cpe_model_os_type      100001 non-null  category
 7   price                  99629 non-null   float64 
 8   date                   100001 non-null  object  
 9   part_of_day            100001 non-null  category
 10  request_cnt            100001 non-null  int8    
 11  user_id                100001 non-null  int32   
 12  age                    100001 non-null  float64 
 13  is_male                100001 non-null  object  
dtypes: category(7), floa

In [3]:
data_features['is_male'] = data_features['is_male'].replace(to_replace=['NA'], value=[np.nan])
print(data_features['is_male'].value_counts())
data_features = data_features.dropna(subset=['is_male'])
data_features['is_male'] = data_features['is_male'].astype('int16')

1    50874
0    45934
Name: is_male, dtype: int64


### Эта залупа работает

In [4]:
%%time
col_name = list(data_features.columns[:7])#
#col_name.remove('user_id')
print(col_name)

def func(column):
    try:
        row = column.value_counts().idxmax()
    except:
        row = np.nan
    return row


table_reg_name = pd.pivot_table(data_features, index=col_name[0], values='is_male', aggfunc=[np.sum, 'count'])
table_reg_name[col_name[0]] = table_reg_name.index
print(table_reg_name.head(5),'\n\n')

#lambda x: x.value_counts().idxmax()
#lambda x: x.mode().iat[0]

print(list(table_reg_name.columns),'\n\n')

print(table_reg_name['sum']['is_male'],'\n\n')

['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name', 'url_host', 'cpe_type_cd', 'cpe_model_os_type']
                          sum   count            region_name
                      is_male is_male                       
region_name                                                 
Алтайский край              4     420         Алтайский край
Амурская область           79      79       Амурская область
Архангельская область       0    1597  Архангельская область
Астраханская область        0       0   Астраханская область
Белгородская область        0       0   Белгородская область 


[('sum', 'is_male'), ('count', 'is_male'), ('region_name', '')] 


region_name
Алтайский край             4
Амурская область          79
Архангельская область      0
Астраханская область       0
Белгородская область       0
                        ... 
Чеченская Республика       0
Чувашская Республика       0
Чукотский АО               0
Ямало-Ненецкий АО          0
Ярославская область 

In [53]:
def func(column):
    try:
        row = (table_reg_name[table_reg_name[col_name[0]].isin(column.unique())]['sum']['is_male'] / table_reg_name[table_reg_name[col_name[0]].isin(column.unique())]['count']['is_male']).mean()
        #print(row)
    except:
        row = np.nan
    return row

table_id = pd.pivot_table(data_features, index='user_id', values=col_name[0], aggfunc=func)
table_id

,region_name
user_id,
5409,0.953403
7004,0.953603
7470,0.527223
9410,0.175389
13248,0.367916
...,...
389612,0.000000
394147,0.816644
401042,0.503893


In [37]:
table['user_id'] = table.index
print(table.index[:10])
print(display(table.loc[:100,:]))



Int64Index([4, 16, 18, 26, 29, 61, 63, 64, 92, 125], dtype='int64', name='user_id')


,age,city_name,cpe_manufacturer_name,cpe_model_name,cpe_model_os_type,cpe_type_cd,date,is_male,part_of_day,price,region_name,request_cnt,url_host,user_id
user_id,,,,,,,,,,,,,,
4,28.0,Анапа,Huawei,Nova 3,Android,smartphone,2021-07-24,0,morning,12990.0,Краснодарский край,1,vk.com,4
16,63.0,Магнитогорск,Samsung,Galaxy A10 Dual,Android,smartphone,2021-07-04,0,day,9583.0,Челябинская область,1,yandex.ru,16
18,50.0,Горячий Ключ,Samsung,Galaxy A51 Dual,Android,smartphone,2021-08-05,1,day,22887.0,Краснодарский край,1,yandex.ru,18
26,59.0,Санкт-Петербург,Samsung,Galaxy A7 2017 Dual,Android,smartphone,2021-06-19,0,evening,4990.0,Санкт-Петербург,1,googleads.g.doubleclick.net,26
29,30.0,Люберцы,Apple,iPhone 8 Plus,iOS,smartphone,2021-07-30,0,day,45696.0,Московская область,1,instagram.com,29
61,25.0,Туапсе,Apple,iPhone XR,iOS,smartphone,2022-02-27,0,evening,49572.0,Краснодарский край,1,apple.com,61
63,29.0,Апатиты,Huawei,Honor 9 Lite,Android,smartphone,2021-07-08,1,day,3990.0,Мурманская область,1,vk.com,63
64,52.0,Оренбург,Samsung,Galaxy J6 Plus Dual,Android,smartphone,2021-06-28,0,morning,1690.0,Оренбургская область,1,yandex.ru,64
92,35.0,Ачинск,Samsung,Galaxy A51 Dual,Android,smartphone,2021-06-19,0,day,20239.0,Красноярский край,1,vk.com,92


None


In [ ]:
print(data_all[data_all['price'].isna()])
print(data_all[data_all['user_id'] == data_all[data_all['price'].isna()]['user_id'].values[0]]['price'].mode().values[0])

In [8]:
del data_all

<p style = 'color:red; font-size:30px;'> Создаём датасет для возраста</p>

In [14]:
col_name = 'cpe_type_cd'

In [15]:
data_train = dd.read_parquet('Dataset\\train test\\train_data_pqt.parquet', columns=[col_name, 'age']).compute()
data_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210730009 entries, 0 to 210730008
Data columns (total 2 columns):
 #   Column       Non-Null Count      Dtype   
---  ------       --------------      -----   
 0   cpe_type_cd  210730009 non-null  category
 1   age          210730009 non-null  int8    
dtypes: category(1), int8(1)
memory usage: 401.9 MB


In [16]:
def age_bucket(x):
    return bisect.bisect_left([25,35,45,55,65], x)

def func(row):
    sum = 0
    for i in range(6):
        sum += row[i]*i
    return sum / np.sum(row)

In [17]:
%%time
data_train = data_train.dropna(subset=['age'])
data_train = data_train.dropna(subset=[col_name])
data_train[col_name] = data_train[col_name].astype('category')

data_train['basket_age'] = data_train['age'].map(age_bucket)

piv_table= pd.pivot_table(data_train, index=col_name, values = 'basket_age', columns='basket_age', aggfunc=['count'])
del data_train
print(display(piv_table))

piv_table.columns = piv_table.columns.droplevel(level=0)
print(piv_table)

piv_table[col_name] = piv_table[[0,1,2,3,4,5]].apply(lambda row : func(row), axis=1)
print(piv_table)

piv_table[col_name] = piv_table.index
piv_table = piv_table.rename(columns={0: 'bask_0', 1: 'bask_1', 2: 'bask_2', 3: 'bask_3', 4: 'bask_4', 5: 'bask_5'})
#piv_table = piv_table.reset_index(drop=True)
print(piv_table)
piv_table.to_parquet(f'Dataset\\age_data\\prob_{col_name}.parquet', index=False)
del piv_table

count                                                 
basket_age          0         1         2         3         4        5
cpe_type_cd                                                           
smartphone   33150262  74707059  56482483  28848696  14566138  2901607
plain             465       529      3892      9160      5706      779
tablet            261      3810     11471      8178     10246     2078
phablet            11       874       928      5304      6731     3341

None
basket_age          0         1         2         3         4        5
cpe_type_cd                                                           
smartphone   33150262  74707059  56482483  28848696  14566138  2901607
plain             465       529      3892      9160      5706      779
tablet            261      3810     11471      8178     10246     2078
phablet            11       874       928      5304      6731     3341
basket_age          0         1         2         3         4        5  \
cpe_type_cd                                                              
smartphone   33150262  74707059  56482483  28848696  14566138  2901607   
plain             465       529      3892      9160      5706      779   
tablet            261      3810     11471      8178     10246     2078   
phablet            11       874       928      5304      6731     3341   

basket_age   cpe_type_cd  
cpe_type_cd               
smartphone      1.647189  
plain           3.044762  
tablet          

In [18]:
%%time

path = 'Dataset\\train test\\'
path_col_data = f'Dataset\\age_data\\prob_{col_name}.parquet'
#mean_age_for_cpe_type.parquet
#mean_age_for_url.parquet
data_all = dd.read_parquet(path+'train_data_pqt.parquet', columns=[col_name, 'user_id']).compute()
#data_all = data_all[:100000]

#test_data_pqt.parquet
#train_data_pqt.parquet
print(data_all.columns)

data_all = data_all.dropna(subset=[col_name])

data_all[col_name] = data_all[col_name].astype('category')
data_all.info(show_counts=True)

data_basket = dd.read_parquet(path_col_data).compute()
data_basket.head(5)

data_all = data_all.merge(data_basket, on = col_name, how = 'inner')
data_all.info(show_counts=True)
print(len(data_all['user_id'].unique()))


piv = pd.pivot_table(data_all, index='user_id', values=['bask_0','bask_1', 'bask_2', 'bask_3', 'bask_4', 'bask_5'], aggfunc=np.sum)
del data_all
piv['user_id'] = piv.index
piv = piv.reset_index(drop=True)
#print(piv)

piv['sum'] = piv['bask_0'] + piv['bask_1'] + piv['bask_2'] + piv['bask_3'] + piv['bask_4'] + piv['bask_5']# + piv['bask_6']
#print(piv)

piv.iloc[:, :6] = piv.iloc[:, :6].div(piv['sum'], axis = 0)
piv = piv.drop(columns=['sum'])
piv.to_parquet(f'Dataset\\age_data\\basket_prob_{col_name}_train.parquet', index=False)
print(display(piv))

del piv


Index(['cpe_type_cd', 'user_id'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 210730009 entries, 0 to 210730008
Data columns (total 2 columns):
 #   Column       Non-Null Count      Dtype   
---  ------       --------------      -----   
 0   cpe_type_cd  210730009 non-null  category
 1   user_id      210730009 non-null  int32   
dtypes: category(1), int32(1)
memory usage: 2.6 GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 210730009 entries, 0 to 210730008
Data columns (total 8 columns):
 #   Column       Non-Null Count      Dtype   
---  ------       --------------      -----   
 0   cpe_type_cd  210730009 non-null  category
 1   user_id      210730009 non-null  int32   
 2   bask_0       210730009 non-null  int64   
 3   bask_1       210730009 non-null  int64   
 4   bask_2       210730009 non-null  int64   
 5   bask_3       210730009 non-null  int64   
 6   bask_4       210730009 non-null  int64   
 7   bask_5       210730009 non-null  int64   
dtypes: c

,bask_0,bask_1,bask_2,bask_3,bask_4,bask_5,user_id
0,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,0
1,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,1
2,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,2
3,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,3
4,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,4
...,...,...,...,...,...,...,...
269994,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415308
269995,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415309
269996,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415311
269997,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415312


None
CPU times: total: 35.5 s
Wall time: 2min


In [19]:
%%time

path = 'Dataset\\train test\\'
path_col_data = f'Dataset\\age_data\\prob_{col_name}.parquet'
#mean_age_for_cpe_type.parquet
#mean_age_for_url.parquet
data_all = dd.read_parquet(path+'test_data_pqt.parquet', columns=[col_name, 'user_id']).compute()
#data_all = data_all[:100000]

#test_data_pqt.parquet
#train_data_pqt.parquet
print(data_all.columns)

data_all = data_all.dropna(subset=[col_name])

data_all[col_name] = data_all[col_name].astype('category')
data_all.info(show_counts=True)

data_basket = dd.read_parquet(path_col_data).compute()
data_basket.head(5)

data_all = data_all.merge(data_basket, on = col_name, how = 'inner')
data_all.info(show_counts=True)
print(len(data_all['user_id'].unique()))


piv = pd.pivot_table(data_all, index='user_id', values=['bask_0','bask_1', 'bask_2', 'bask_3', 'bask_4', 'bask_5'], aggfunc=np.sum)
del data_all
piv['user_id'] = piv.index
piv = piv.reset_index(drop=True)
#print(piv)

piv['sum'] = piv['bask_0'] + piv['bask_1'] + piv['bask_2'] + piv['bask_3'] + piv['bask_4'] + piv['bask_5']# + piv['bask_6']
#print(piv)

piv.iloc[:, :6] = piv.iloc[:, :6].div(piv['sum'], axis = 0)
piv = piv.drop(columns=['sum'])
piv.to_parquet(f'Dataset\\age_data\\basket_prob_{col_name}_test.parquet', index=False)
print(display(piv))

del piv


Index(['cpe_type_cd', 'user_id'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 111734107 entries, 0 to 111734106
Data columns (total 2 columns):
 #   Column       Non-Null Count      Dtype   
---  ------       --------------      -----   
 0   cpe_type_cd  111734107 non-null  category
 1   user_id      111734107 non-null  int64   
dtypes: category(1), int64(1)
memory usage: 1.8 GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 111734107 entries, 0 to 111734106
Data columns (total 8 columns):
 #   Column       Non-Null Count      Dtype   
---  ------       --------------      -----   
 0   cpe_type_cd  111734107 non-null  category
 1   user_id      111734107 non-null  int64   
 2   bask_0       111734107 non-null  int64   
 3   bask_1       111734107 non-null  int64   
 4   bask_2       111734107 non-null  int64   
 5   bask_3       111734107 non-null  int64   
 6   bask_4       111734107 non-null  int64   
 7   bask_5       111734107 non-null  int64   
dtypes: c

,bask_0,bask_1,bask_2,bask_3,bask_4,bask_5,user_id
0,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,6
1,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,7
2,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,9
3,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,10
4,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,11
...,...,...,...,...,...,...,...
144719,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415306
144720,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415310
144721,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415314
144722,0.157367,0.35464,0.268126,0.136947,0.069146,0.013774,415315


None
CPU times: total: 37.2 s
Wall time: 1min 6s


In [6]:
%%time

def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

def func(row):
    sum = 0
    for i in range(7):
        sum += row[i]*i
    return sum / np.sum(row)


col_name = 'cpe_type_cd'

path = 'Dataset\\train test\\'
data_all = dd.read_parquet(path+'train_data_pqt.parquet', columns=[col_name,'age']).compute()
print(data_all.columns)

data_all = data_all.dropna(subset=['age'])
data_all = data_all.dropna(subset=[col_name])

data_all[col_name] = data_all[col_name].astype('category')
print(data_all.info(show_counts=True))


data_all['basket_age'] = data_all['age'].map(age_bucket)
piv_count = pd.pivot_table(data_all, index=col_name, values = 'basket_age', columns='basket_age', aggfunc=['count'])
del data_all
print(display(piv_count))

piv_count.columns = piv_count.columns.droplevel(level=0)
print(piv_count)


piv_count[col_name] = piv_count[[0,1,2,3,4,5,6]].apply(lambda row : func(row), axis=1)
print(piv_count)

piv_count[col_name] = piv_count.index
piv_count = piv_count.rename(columns={0: 'bask_0', 1: 'bask_1', 2: 'bask_2', 3: 'bask_3', 4: 'bask_4', 5: 'bask_5', 6: 'bask_6'})
print(piv_count)


piv_count.to_parquet('Dataset\\age_data\\mean_age_for_cpe_type.parquet', index=False)
#mean_age_for_cpe_type
#mean_age_for_url

Index(['cpe_type_cd', 'age'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210730009 entries, 0 to 210730008
Data columns (total 2 columns):
 #   Column       Non-Null Count      Dtype   
---  ------       --------------      -----   
 0   cpe_type_cd  210730009 non-null  category
 1   age          210730009 non-null  int8    
dtypes: category(1), int8(1)
memory usage: 401.9 MB
None


count                                                           
basket_age        0         1         2         3         4         5        6
cpe_type_cd                                                                   
smartphone   826300  32323962  74707059  56482483  28848696  14566138  2901607
plain             0       465       529      3892      9160      5706      779
tablet            0       261      3810     11471      8178     10246     2078
phablet           0        11       874       928      5304      6731     3341

None
basket_age        0         1         2         3         4         5        6
cpe_type_cd                                                                   
smartphone   826300  32323962  74707059  56482483  28848696  14566138  2901607
plain             0       465       529      3892      9160      5706      779
tablet            0       261      3810     11471      8178     10246     2078
phablet           0        11       874       928      5304      6731     3341
basket_age        0         1         2         3         4         5  \
cpe_type_cd                                                             
smartphone   826300  32323962  74707059  56482483  28848696  14566138   
plain             0       465       529      3892      9160      5706   
tablet            0       261      3810     11471      8178     10246   
phablet           0        11       874       928      5304      6731   

basket_age         6 cpe_type_cd  
cpe_type_cd                       
smartphone  

In [2]:
%%time
col_name = 'url_host'
#cpe_type_cd
#url_host

path = 'Dataset\\train test\\'
path_col_data = 'Dataset\\age_data\\mean_age_for_url.parquet'
#mean_age_for_cpe_type.parquet
#mean_age_for_url.parquet



data_all = dd.read_parquet(path+'test_data_pqt.parquet', columns=[col_name, 'user_id']).compute()


#test_data_pqt.parquet
#train_data_pqt.parquet
print(data_all.columns)

data_all = data_all.dropna(subset=[col_name])

data_all[col_name] = data_all[col_name].astype('category')
data_all.info(show_counts=True)

data_basket = dd.read_parquet(path_col_data).compute()
data_basket.head(5)

data_all = data_all.merge(data_basket, on = col_name, how = 'inner')
data_all.info(show_counts=True)
print(len(data_all['user_id'].unique()))

def func(x):
    sum_col = x.sum()
    sum_all = sum_col.sum()
    res = sum_col / sum_all
    res = pd.Series(res)
    res = res.to_frame('prob')
    return res.T.to_numpy()[0]

piv = pd.pivot_table(data_all, index='user_id', values=['bask_0','bask_1', 'bask_2', 'bask_3', 'bask_4', 'bask_5','bask_6'], aggfunc=func)
piv['user_id'] = piv.index
piv.to_parquet(f'Dataset\\age_data\\basket_prob_{col_name}_test.parquet', index=False)
#basket_prob_cpe_type_train.parquet
#basket_prob_url_train.parquet
print(display(piv))

Index(['url_host', 'user_id'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 111734107 entries, 0 to 44625242
Data columns (total 2 columns):
 #   Column    Non-Null Count      Dtype   
---  ------    --------------      -----   
 0   url_host  111734107 non-null  category
 1   user_id   111734107 non-null  int64   
dtypes: category(1), int64(1)
memory usage: 2.1 GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 111595145 entries, 0 to 111595144
Data columns (total 9 columns):
 #   Column    Non-Null Count      Dtype 
---  ------    --------------      ----- 
 0   url_host  111595145 non-null  object
 1   user_id   111595145 non-null  int64 
 2   bask_0    111595145 non-null  int64 
 3   bask_1    111595145 non-null  int64 
 4   bask_2    111595145 non-null  int64 
 5   bask_3    111595145 non-null  int64 
 6   bask_4    111595145 non-null  int64 
 7   bask_5    111595145 non-null  int64 
 8   bask_6    111595145 non-null  int64 
dtypes: int64(8), object(1)
memor


KeyboardInterrupt



In [45]:
del data_all

NameError: name 'data_all' is not defined

### Тестирование внизу различных предположений

In [6]:
data_target_file = dd.read_parquet('submit_2.pqt').compute()
print(data_target_file.info())
#print(data_target_file[data_target_file['age'].isna()])
del data_target_file

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144724 entries, 221301 to 145315
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  144724 non-null  int64
dtypes: int64(1)
memory usage: 2.2 MB
None


In [15]:
data_features_file = dd.read_parquet('.\\competition_data_final_pqt' + f'\\part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet').compute()

data_target_file = dd.read_parquet('public_train.pqt').compute()

display(data_target_file)

,age,is_male,user_id
350459,31.0,1,350459
188276,35.0,1,188276
99002,41.0,0,99002
155506,33.0,0,155506
213873,54.0,0,213873
...,...,...,...
225374,49.0,1,225374
25776,22.0,1,25776
148131,28.0,0,148131
205570,28.0,1,205570


In [18]:
data_features_file.columns

Index(['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
       'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price', 'date',
       'part_of_day', 'request_cnt', 'user_id'],
      dtype='object')

In [33]:
#del data_features_file
#del data_target_file

In [ ]:
#data_features_file = data_features_file.loc[:100]
display(data_features_file)

In [30]:
data_features_file = data_features_file.merge(data_target_file[['age', 'is_male','user_id']], on = 'user_id', how = 'left')

In [ ]:
data_features_file

In [ ]:
display(data_features_file[data_features_file['age'].notna()])